In [501]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random

In [502]:
data = pd.read_csv('dataset/dataCovidIndonesia.csv')
data = data.loc[0:49, 'Kasus harian']
data = np.genfromtxt(data, dtype= None)
data

array([  2,   0,   0,   0,   2,   0,   2,  13,   8,   0,  35,  27,  21,
        17,  38,  55,  82,  60,  81,  64,  65, 106, 105, 103, 153, 109,
       130, 129, 114, 149, 113, 196, 106, 181, 218, 247, 218, 337, 219,
       330, 399, 316, 282, 297, 380, 407, 325, 327, 185, 375])

In [503]:
def generasi_awal(pop_size, gene_per_pop):
    population_size = (pop_size, gene_per_pop)
    gene = np.random.uniform(low=0.1, high=0.9, size=population_size)
    return gene #generate individu dengan 3 gen pada setiap individu

In [504]:
def day_predict(pop_size, gene_per_pop):
    day = []
    for i in range(2, pop_size):
        row = []
        for j in range(0, gene_per_pop):
            if j == 0:
                row.append(data[i-1]) #y(t-1) 
            elif j == 1:
                row.append(data[i-2]) #y(t-2)
            elif j == 2:
                row.append(data[i]) #y(target)
        day.append(row)
    return day

In [505]:
def y_pred(pop_size, gene, day_predict, gene_per_pop):
    y_pred_indv = []
    for i in range (0,pop_size-2):
        row = []
        for j in gene:
            total = 0
            z = 0
            for k in j:
                if k<gene_per_pop-1:
                    g = k*day_predict[i][z]
                else:
                    g = k
                z=z+1
                total = total + g
            row.append(total)
        y_pred_indv.append(row)
    return y_pred_indv #generate y_pred_individu

In [506]:
def fitness_func(ps, day_predict, hasil_prediksi, gene_per_pop):  
    mape = []
    fitn = []
    for i in range(0,ps):
        row = []
        for j in range(0,len(data_y)):
            if hasil_prediksi[j][i]>0: #hitung mape
                row.append(abs((hasil_prediksi[j][i]-data_y[j])/hasil_prediksi[j][i]))
            else:
                row.append(abs(0)) #mape akan bernilai 0 ketika hasil_prediksi bernilai 0
        mape.append(row)        
    for i in range(0,len(mape)):
        f = 0
        for j in range (0,ps-2):
            f = f + mape[i][j]
        f=f/len(mape)
        fitn.append(1/(f+0.01))
    
    return fitn

In [507]:
def probability(fitness):
    prob = []
    sumF = 0
    for i in range(0,len(fitness)):
        sumF = sumF + fitness[i]
    for i in range(0,len(fitness)):
        prob.append(fitness[i]/sumF*100)
    return prob #generate probabilitas setiap fitness individu

In [508]:
def rangePc(probs):
    ranges = []
    x = 0
    y = 0
    minArray = []
    maxArray = []
    for i in range(0,len(probs)):
        if i == 0:
            x = 0;
        else:
            x = x + probs[i-1]
        y = y + probs[i]
        minArray.append(x)
        maxArray.append(y)
    ranges = np.vstack((minArray, maxArray)).T
    return ranges #generate nilai min dan max pada setiap probabilitas indifidu

In [509]:
def round_wheel(pc,fitness,ranges):
    rW = []
    for i in range(0,int(pc*50)):
        rNum = random.uniform(0,100)
        for j in range(0,len(ranges)):
            if rNum >= ranges[j][0] and rNum <= ranges[j][1]:
                rW.append(j)
    return rW #terpilih individu ke-sekian

In [510]:
def cross_over(pc,rw,gA):
    co = []
    x = 0
    y = 0
    for i in range(0,len(rw)):
        row = []
        for j in range(0,gene_per_pop):
            row.append(gA[rw[i]][j])
        co.append(row) #mendapatkan nilai gen dari hasil roundwheel
    
    for i in range(0,len(rw),2):
        if i % 2 == 0 and i == len(rw)-1:
            co[i][1], co[0][1] = (co[0][1]*(1-pc))+(co[i][1]*pc), (co[i][1]*(1-pc))+(co[0][1]*pc) #swap pasangan akhir ke pasangan awal ketika jumlah datanya ganjil
        else:
            co[i][1], co[i+1][1] = (co[i+1][1]*(1-pc))+(co[i][1]*pc), (co[i][1]*(1-pc))+(co[i+1][1]*pc)
    return co #menghasilkan nilai offspring berdasarkan hasil crossover

In [511]:
def evalGen(generasiAwal,gene_per_pop,fitness,offSpring,fitnessOffspring):
    evalIndv = []
    indvAwal = []
    indvOffspring = []
    for i in range(0,len(generasiAwal)): 
        row = []
        for j in range(0,gene_per_pop+1):
            if j == gene_per_pop:
                row.append(fitness[i])
            else:
                row.append(generasiAwal[i][j])
        indvAwal.append(row) #membuat matrix berisi 3gen individu dan fitness dari generasi awal
    for i in range(0,len(offSpring)):
        row = []
        for j in range(0,gene_per_pop+1):
            if j == gene_per_pop:
                row.append(fitnessOffspring[i])
            else:
                row.append(offSpring[i][j])
        indvOffspring.append(row) #membuat matrix berisi 3gen individu dan fitness dari offspring
    evalIndv = indvAwal + indvOffspring #menggabungkan 2 matrix untuk dapat di sorting berdasarkan fitness
    return sorted(evalIndv, key=lambda x: x[3], reverse=True) #sorting descending berdasarkan nilai fitness

In [512]:
def eval_elitism(evalIndividu,pop_size):
    for i in range(0,len(evalIndividu)-pop_size):
        evalIndividu.pop() #mengeliminasi individu yang memiliki fitness terendah
    return evalIndividu

In [513]:
pop_size = 50
gene_per_pop = 3
pc = 0.7
generasiAwal = generasi_awal(pop_size, gene_per_pop)
generasiAwal

array([[0.20100129, 0.45161837, 0.23539759],
       [0.73458658, 0.59233468, 0.59511946],
       [0.61621093, 0.36878851, 0.86817619],
       [0.72844979, 0.59695834, 0.87030675],
       [0.46332861, 0.53651471, 0.53074867],
       [0.19529048, 0.5264975 , 0.25175933],
       [0.28011386, 0.59047324, 0.67474899],
       [0.28493921, 0.79880861, 0.57205694],
       [0.30391723, 0.44286914, 0.44564299],
       [0.88616694, 0.19549818, 0.7028704 ],
       [0.86896408, 0.75952956, 0.39680227],
       [0.75722474, 0.11824519, 0.158181  ],
       [0.88235989, 0.80485865, 0.76400626],
       [0.74408859, 0.10002829, 0.58837712],
       [0.54905573, 0.24971934, 0.40511069],
       [0.38048915, 0.49918742, 0.36823195],
       [0.1919825 , 0.71841265, 0.89026128],
       [0.43195219, 0.43883468, 0.53695451],
       [0.75066285, 0.87409219, 0.50803078],
       [0.52281334, 0.75270186, 0.25809761],
       [0.87381987, 0.86797051, 0.22244174],
       [0.7403421 , 0.37622204, 0.69328976],
       [0.

In [514]:
dayPredict = day_predict(pop_size, gene_per_pop)
dayPredict

[[0, 2, 0],
 [0, 0, 0],
 [0, 0, 2],
 [2, 0, 0],
 [0, 2, 2],
 [2, 0, 13],
 [13, 2, 8],
 [8, 13, 0],
 [0, 8, 35],
 [35, 0, 27],
 [27, 35, 21],
 [21, 27, 17],
 [17, 21, 38],
 [38, 17, 55],
 [55, 38, 82],
 [82, 55, 60],
 [60, 82, 81],
 [81, 60, 64],
 [64, 81, 65],
 [65, 64, 106],
 [106, 65, 105],
 [105, 106, 103],
 [103, 105, 153],
 [153, 103, 109],
 [109, 153, 130],
 [130, 109, 129],
 [129, 130, 114],
 [114, 129, 149],
 [149, 114, 113],
 [113, 149, 196],
 [196, 113, 106],
 [106, 196, 181],
 [181, 106, 218],
 [218, 181, 247],
 [247, 218, 218],
 [218, 247, 337],
 [337, 218, 219],
 [219, 337, 330],
 [330, 219, 399],
 [399, 330, 316],
 [316, 399, 282],
 [282, 316, 297],
 [297, 282, 380],
 [380, 297, 407],
 [407, 380, 325],
 [325, 407, 327],
 [327, 325, 185],
 [185, 327, 375]]

In [515]:
yPred = y_pred(pop_size, generasiAwal, dayPredict, gene_per_pop)
yPred

[[0.9032367379389556,
  1.1846693562809407,
  0.7375770139246434,
  1.1939166791910547,
  1.0730294167551973,
  1.0529949910680878,
  1.1809464891007484,
  1.5976172196339467,
  0.8857382879700029,
  0.3909963537851107,
  1.5190591228037689,
  0.23649038753482704,
  1.6097172974889453,
  0.2000565720528938,
  0.4994386801886155,
  0.9983748440586311,
  1.4368252900062284,
  0.8776693574913732,
  1.7481843780858124,
  1.5054037298942036,
  1.7359410203504697,
  0.7524440725427401,
  0.6344509707621295,
  1.6310509766370256,
  1.2115736127699606,
  1.3719100098504662,
  0.8935715035100138,
  1.6042988846436939,
  1.2017096791073905,
  0.4499153131758865,
  0.21077993792466485,
  1.6711295341049366,
  1.1585003416445583,
  0.3712750938647144,
  0.5479825136979524,
  1.2182736570500357,
  0.2246270956530026,
  0.6248470565146047,
  0.8156891288028472,
  0.43165611841602464,
  1.684292172086925,
  1.1770757893264738,
  0.8576472329272213,
  0.7194392024242455,
  0.54801400836477,
  1.050163

In [516]:
dayPredict = np.array(dayPredict)
data_y = dayPredict[:, (gene_per_pop-1)]
fitness = fitness_func(pop_size, day_predict, yPred, gene_per_pop)
fitness

[2.1412819563105923,
 2.1811938486829425,
 2.3246848477367363,
 2.015598459462611,
 2.8208828459758104,
 2.466947828429199,
 2.902033790281607,
 2.604392786667611,
 3.738929123830947,
 2.351741037216694,
 1.9781946055044077,
 1.7935645890372818,
 1.8589424190148,
 2.9688544141705155,
 3.3862645937161586,
 3.2343866074164778,
 2.4438257105379284,
 3.1999260599103723,
 1.9732694566437103,
 2.3326892307297826,
 1.8102901423974838,
 2.340059413817386,
 3.07505309089014,
 2.6905795609411607,
 3.419122356287139,
 1.8880918264523001,
 2.0460341070457972,
 2.069933649010042,
 3.338255410959862,
 2.7852441001984154,
 3.1833619733829575,
 2.4739379955328986,
 2.149345093076648,
 2.688803008783286,
 2.2861912197815712,
 1.9707978205863046,
 3.843460800508134,
 2.4424245644503166,
 2.3717881057607295,
 2.781981797004377,
 2.6901720040159636,
 2.81754532037091,
 2.310959205845142,
 5.159272240724796,
 2.4463203282320407,
 3.507977174685366,
 2.370787106903505,
 2.5318269657498296,
 1.95667248091681

In [517]:
yProb = probability(fitness)
yProb

[1.6245384884852463,
 1.6548186695310343,
 1.7636817970734384,
 1.52918547932397,
 2.140135137873825,
 1.8716132569825024,
 2.2017023836133056,
 1.9758894005555163,
 2.8366344980776352,
 1.7842086693204136,
 1.500808085962155,
 1.3607337874810237,
 1.410334355392743,
 2.2523975641392573,
 2.5690765050694804,
 2.453850374493672,
 1.854071069070742,
 2.4277059960791636,
 1.4970714954294875,
 1.7697545275751168,
 1.3734230576141724,
 1.7753461060488005,
 2.3329679146476456,
 2.0412772078889394,
 2.594004889575199,
 1.432449301142507,
 1.5522762641573933,
 1.57040826478611,
 2.5326531069464666,
 2.1130968891153694,
 2.4151392269009704,
 1.8769165265810888,
 1.6306557940440842,
 2.039929381018796,
 1.7344776187118258,
 1.4951963253273204,
 2.9159401362387003,
 1.8530080536464502,
 1.7994178921577464,
 2.1106218591062644,
 2.0409680043721954,
 2.1376030384529683,
 1.7532684867355135,
 3.9142142411138066,
 1.8559636747811665,
 2.6614168770296662,
 1.7986584586951433,
 1.92083547891664,
 1.484

In [518]:
yRanges = rangePc(yProb)
yRanges

array([[  0.        ,   1.62453849],
       [  1.62453849,   3.27935716],
       [  3.27935716,   5.04303896],
       [  5.04303896,   6.57222443],
       [  6.57222443,   8.71235957],
       [  8.71235957,  10.58397283],
       [ 10.58397283,  12.78567521],
       [ 12.78567521,  14.76156461],
       [ 14.76156461,  17.59819911],
       [ 17.59819911,  19.38240778],
       [ 19.38240778,  20.88321587],
       [ 20.88321587,  22.24394965],
       [ 22.24394965,  23.65428401],
       [ 23.65428401,  25.90668157],
       [ 25.90668157,  28.47575808],
       [ 28.47575808,  30.92960845],
       [ 30.92960845,  32.78367952],
       [ 32.78367952,  35.21138552],
       [ 35.21138552,  36.70845701],
       [ 36.70845701,  38.47821154],
       [ 38.47821154,  39.8516346 ],
       [ 39.8516346 ,  41.62698071],
       [ 41.62698071,  43.95994862],
       [ 43.95994862,  46.00122583],
       [ 46.00122583,  48.59523072],
       [ 48.59523072,  50.02768002],
       [ 50.02768002,  51.57995628],
 

In [519]:
roundWheel = round_wheel(pc,fitness,yRanges)
roundWheel

[34,
 39,
 7,
 30,
 49,
 47,
 14,
 49,
 29,
 8,
 23,
 15,
 5,
 36,
 7,
 45,
 22,
 15,
 17,
 15,
 47,
 48,
 49,
 22,
 38,
 15,
 17,
 47,
 14,
 14,
 6,
 49,
 31,
 19,
 43]

In [520]:
offSpring = cross_over(pc,roundWheel,generasiAwal)
offSpring

[[0.8527626679315826, 0.28749548865331775, 0.7327292306397875],
 [0.7448855081771336, 0.23327701850030147, 0.5315560992159016],
 [0.28493920663000916, 0.590783017560581, 0.5720569386086419],
 [0.7206384484862182, 0.3134155612187247, 0.49942571884419873],
 [0.2741935646823933, 0.35311746637698743, 0.7155531644317292],
 [0.7570446579394509, 0.3399572182288911, 0.5398723257132826],
 [0.5490557292048479, 0.28369983381243336, 0.40511069054510285],
 [0.2741935646823933, 0.32900715876993414, 0.7155531644317292],
 [0.759895418493714, 0.29033110280706076, 0.4742472827337696],
 [0.30391722836371005, 0.377495697765884, 0.44564299237781524],
 [0.24446555565633227, 0.7206240684317535, 0.5317278417374429],
 [0.3804891473524069, 0.5940888419160748, 0.36823194529622216],
 [0.1952904759526919, 0.4022423112217811, 0.25175932917675575],
 [0.4377908078377375, 0.23656873213876412, 0.7449237079111414],
 [0.28493920663000916, 0.7166905851910066, 0.5720569386086419],
 [0.25597710236883675, 0.6071998856897177,

In [521]:
yPredOffspring = y_pred(pop_size, offSpring, dayPredict, gene_per_pop)
len(yPredOffspring)

48

In [522]:
fitnessOffspring = fitness_func(len(yPredOffspring[0]), day_predict, yPredOffspring, gene_per_pop)
fitnessOffspring

[2.3228284984874117,
 2.640003310869703,
 3.061670254773438,
 2.5853812734751247,
 3.6203017922346254,
 2.4751286487390494,
 2.8851215128308776,
 3.7399380394487487,
 2.5345917140712557,
 3.303459550898502,
 2.855382633199487,
 2.7076589479623445,
 1.7271434353458337,
 3.2316625435397035,
 2.7703633123658244,
 3.0975109535049925,
 2.9505320186672357,
 2.828594838007991,
 3.0409947288148396,
 2.8028905073703623,
 2.359042959306688,
 1.6691983424553947,
 3.638652969559275,
 3.0544164915796665,
 2.1179477015736836,
 2.809208798151979,
 3.1519607743086593,
 2.4493248858364822,
 2.904557757011165,
 2.904557757011165,
 3.1023087553541355,
 3.302155996998425,
 2.5846386513586292,
 2.1015649623807984,
 4.465250900861673]

In [523]:
evalIndividu = evalGen(generasiAwal,gene_per_pop,fitness,offSpring,fitnessOffspring)
evalIndividu

[[0.11090389118237748,
  0.35971960121212276,
  0.6314856029915521,
  5.159272240724796],
 [0.11090389118237748,
  0.328766410115492,
  0.6314856029915521,
  4.465250900861673],
 [0.4377908078377375,
  0.1123135478265013,
  0.7449237079111414,
  3.843460800508134],
 [0.2741935646823933,
  0.32900715876993414,
  0.7155531644317292,
  3.7399380394487487],
 [0.30391722836371005,
  0.44286914398500143,
  0.44564299237781524,
  3.738929123830947],
 [0.2741935646823933,
  0.36298765248805975,
  0.7155531644317292,
  3.644738821288533],
 [0.2741935646823933,
  0.34925900235596125,
  0.7155531644317292,
  3.638652969559275],
 [0.2741935646823933,
  0.35311746637698743,
  0.7155531644317292,
  3.6203017922346254],
 [0.25597710236883675,
  0.525081861063751,
  0.5641704256571499,
  3.507977174685366],
 [0.13495194234797658,
  0.6057868063849803,
  0.6477298961025573,
  3.419122356287139],
 [0.5490557292048479,
  0.24971934009430774,
  0.40511069054510285,
  3.3862645937161586],
 [0.1141041867010

In [524]:
evalElitism = eval_elitism(evalIndividu,pop_size)
evalElitism

[[0.11090389118237748,
  0.35971960121212276,
  0.6314856029915521,
  5.159272240724796],
 [0.11090389118237748,
  0.328766410115492,
  0.6314856029915521,
  4.465250900861673],
 [0.4377908078377375,
  0.1123135478265013,
  0.7449237079111414,
  3.843460800508134],
 [0.2741935646823933,
  0.32900715876993414,
  0.7155531644317292,
  3.7399380394487487],
 [0.30391722836371005,
  0.44286914398500143,
  0.44564299237781524,
  3.738929123830947],
 [0.2741935646823933,
  0.36298765248805975,
  0.7155531644317292,
  3.644738821288533],
 [0.2741935646823933,
  0.34925900235596125,
  0.7155531644317292,
  3.638652969559275],
 [0.2741935646823933,
  0.35311746637698743,
  0.7155531644317292,
  3.6203017922346254],
 [0.25597710236883675,
  0.525081861063751,
  0.5641704256571499,
  3.507977174685366],
 [0.13495194234797658,
  0.6057868063849803,
  0.6477298961025573,
  3.419122356287139],
 [0.5490557292048479,
  0.24971934009430774,
  0.40511069054510285,
  3.3862645937161586],
 [0.1141041867010